In [1]:
import csv
import re
import pandas as pd
fg_pattern = re.compile(r'(\w\. [a-z]+)? makes (2-pt|3-pt)', re.I) 
fga_pattern = re.compile(r'(\w\. [a-z]+)? \w+ (2-pt|3-pt)', re.I)
p3_pattern = re.compile(r'(\w\. [a-z]+)? makes 3-pt', re.I)
p3a_pattern = re.compile(r'(\w\. [a-z]+)? \w+ 3-pt', re.I) 
ft_pattern = re.compile(r'(\w\. [a-z]+)? makes ([a-z ]+)? ?free throw', re.I) 
fta_pattern = re.compile(r'(\w\. [a-z]+)? \w+ ([a-z ]+)? ?free throw (\d) of (\d)', re.I)
orb_pattern = re.compile(r'Offensive rebound \w+ (\w\. [a-z]+)', re.I)
drb_pattern = re.compile(r'Defensive rebound \w+ (\w\. [a-z]+)', re.I)
ast_pattern = re.compile(r'assist by (\w\. [a-z]+)', re.I)
stl_pattern = re.compile(r'steal by (\w\. [a-z]+)', re.I)
blk_pattern = re.compile(r'block by (\w\. [a-z]+)', re.I) 
tov_pattern = re.compile(r'turnover by (\w\. [a-z]+)', re.I) 
pf_pattern = re.compile(r'foul by (\w\. [a-z]+)', re.I)
missed_pattern = re.compile(r'(\w\. \w+ misses| J. Nurkić )', re.I)

In [2]:
#Loading data and creating and making it 2d list
def load_data(file):
    play_by_play = list()
    with open(file,'r') as csvfile:
        csvreader = csv.reader(csvfile, delimiter = "|")
        
        for row in csvreader:
            play_by_play.append(row)
    return play_by_play


#Making the list of all players from the data
def find_all_the_players(play_by_play):
    players_list = []
    player_name = re.compile(r'\w\. \w+', re.I)
    for event in play_by_play:
        name = player_name.search(event[-1])
        if name:
            players_list.append(name.group(0)) 
    return list(set(players_list)) 



In [3]:

def analyse_nba_game(play_by_play):

    players_data = {}
    for i in players:
        players_data[i] = {"FG": 0, "FGA": 0, "FG%": 0, "3P": 0, "3PA": 0, "3P%": 0, "FT": 0, "FTA": 0, "FT%": 0, "ORB": 0, "DRB": 0, "TRB": 0, "AST": 0, "STL": 0, "BLK": 0, "TOV": 0, "PF": 0, "PTS": 0}
    
    home_team = []
    away_team = []

    for player in players:
        for event in play_by_play: 
            pf = pf_pattern.search(event[-1])
            missed = missed_pattern.search(event[-1])
            drb = drb_pattern.search(event[-1])
            fga = fga_pattern.search(event[-1]) 
            if fga:
                if fga.group(1) == player:
                    if event[2] == event[3]:
                        away_team.append(player)
                    elif event[2]== event[4]:
                        home_team.append(player)
                    break
            if missed:
                if missed.group(1) == player:
                    if event[2] == event[3]:
                        away_team.append(player)
                    elif event[2]== event[4]:
                        home_team.append(player)
                    break
                    
            if drb:
                if drb.group(1) == player:
                    if event[2] == event[3]:
                        away_team.append(player)
                    elif event[2]== event[4]:
                        home_team.append(player)
                    break

            if pf:
                if pf.group(1) == player:
                    if event[2] == event[3]:
                        home_team.append(player)
                    elif event[2]== event[4]:
                        away_team.append(player)
                    break
    


    for player in players:
         for event in play_by_play:     
                fga = fga_pattern.search(event[-1]) 
                fg = fg_pattern.search(event[-1])
                p3 = p3_pattern.search(event[-1])
                p3a = p3a_pattern.search(event[-1])
                if fga:
                    if fga.group(1) == player:
                        players_data[player]['FGA'] += 1 
                        if fg:
                            players_data[player]['FG'] += 1
                        if p3a:
                            players_data[player]['3PA'] += 1
                            if p3:
                                players_data[player]['3P'] += 1
                
                fta = fta_pattern.search(event[-1]) 
                ft = ft_pattern.search(event[-1]) 
                if fta:
                    if fta.group(1) == player:
                        players_data[player]['FTA'] += 1
                        if ft:
                            players_data[player]['FT'] += 1
                            
                orb = orb_pattern.search(event[-1]) 
                drb = drb_pattern.search(event[-1])
                ast = ast_pattern.search(event[-1])
                stl = p3a_pattern.search(event[-1])
                blk = blk_pattern.search(event[-1])
                tov = tov_pattern.search(event[-1])
                pf = pf_pattern.search(event[-1])
               
                
                if pf:
                    if pf.group(1) == player:
                        players_data[player]['PF'] += 1
                
                if orb:
                    if orb.group(1) == player:
                        players_data[player]['ORB'] += 1
                
                if drb:
                    if drb.group(1) == player:
                        players_data[player]['DRB'] += 1
                
                if ast:
                    if ast.group(1) == player:
                        players_data[player]['AST'] += 1
                
                if stl:
                    if stl.group(1) == player:
                        players_data[player]['STL'] += 1
                
                if blk:
                    if blk.group(1) == player:
                        players_data[player]['BLK'] += 1
                
                if tov:
                    if tov.group(1) == player:
                        players_data[player]['TOV'] += 1
            
                PS2 = 2*(players_data[player]['FG'] - players_data[player]['3P'])
                FT1 =  players_data[player]['FT']
                
                if players_data[player]['FGA']!=0:
                    players_data[player]['FG%'] = players_data[player]['FG']/players_data[player]['FGA']
                    
                if players_data[player]['3PA']!=0:
                    players_data[player]['3P%'] = players_data[player]['3P']/players_data[player]['3PA']
                
                if players_data[player]['FTA']!=0:
                    players_data[player]['FT%'] = players_data[player]['FT']/players_data[player]['FTA']
                    
                players_data[player]['TRB'] = players_data[player]['DRB']+players_data[player]['ORB']
                
                players_data[player]['PTS'] = 3*(players_data[player]['3P'])+PS2+FT1

  
    players_data_team = {"home_team":{"name":play_by_play[1][4],"players_data":0},"away_team":{"name": play_by_play[1][3],"players_data":0}}
    
    home_team_dict = {j:0 for j in home_team}
    away_team_dict = {j:0 for j in away_team}
    
    for name in home_team_dict:
        home_team_dict[name] = players_data[name]
    
    for name in away_team_dict:
        away_team_dict[name] = players_data[name]
    
    players_data_team["home_team"]["players_data"] =  home_team_dict
    players_data_team["away_team"]["players_data"] =  away_team_dict
 
    return players_data_team




In [8]:
def print_nba_game_stats(team_dict, team):
    player = list(team_dict[team]['players_data'].keys())[0]
    columns =list(team_dict[team]['players_data'][player].keys())
    rows = list(team_dict[team]['players_data'].keys())
    rows.append("Team totals")
    data = {'Players':rows}
    for par in columns:
        data[par] = [0]*len(rows)
    df = pd.DataFrame(data)
    
    for i in range(len(rows)-1):
        name = df.iloc[i][0]
        for par in columns:
            df.loc[i,par] = team_dict[team]['players_data'][name][par]
    for par in columns:
        df.loc[len(rows)-1, par] = df[par].sum()
        
    return df

In [5]:
play_by_play = load_data("nba_game.txt")

players = find_all_the_players(play_by_play)

data = analyse_nba_game(play_by_play)




In [9]:
print_nba_game_stats(data, "home_team")

,Players,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Z. Collins,3,4,0.750000,0,1,0.000000,0,0,0.00,1,5,6,2,1,6,1,5,6
1,M. Leonard,0,0,0.000000,0,0,0.000000,0,0,0.00,1,3,4,4,0,0,1,1,0
2,C. McCollum,6,17,0.352941,3,6,0.500000,6,6,1.00,1,4,5,1,6,0,2,4,21
3,N. Stauskas,7,11,0.636364,5,8,0.625000,5,5,1.00,0,2,2,2,8,0,0,1,24
4,E. Turner,5,8,0.625000,0,0,0.000000,3,4,0.75,1,2,3,6,0,0,3,2,13
5,J. Layman,1,4,0.250000,1,4,0.250000,0,0,0.00,0,2,2,0,4,0,1,1,3
6,D. Lillard,9,21,0.428571,2,7,0.285714,8,8,1.00,1,5,6,4,7,1,3,0,28
7,S. Curry,2,6,0.333333,1,2,0.500000,0,0,0.00,1,4,5,0,2,0,1,2,5
8,M. Harkless,3,5,0.600000,1,2,0.500000,0,0,0.00,2,4,6,2,2,3,1,3,7
9,A. Aminu,1,10,0.100000,0,6,0.000000,3,4,0.75,0,6,6,0,6,0,1,3,5


In [10]:
print_nba_game_stats(data, "away_team")

,Players,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,L. James,9,16,0.562500,0,4,0.000000,8,9,0.888889,2,10,12,6,4,0,6,3,26
1,R. Rondo,6,13,0.461538,1,2,0.500000,0,0,0.000000,0,4,4,11,2,0,3,6,13
2,K. Kuzma,5,15,0.333333,1,7,0.142857,4,5,0.800000,0,5,5,0,7,0,1,3,15
3,M. Beasley,0,0,0.000000,0,0,0.000000,0,0,0.000000,0,1,1,0,0,0,0,0,0
4,K. Caldwell,0,0,0.000000,0,0,0.000000,0,0,0.000000,0,2,2,0,0,0,0,1,0
5,B. Ingram,7,15,0.466667,0,4,0.000000,2,4,0.500000,1,3,4,1,4,1,1,0,16
6,L. Stephenson,2,6,0.333333,0,3,0.000000,0,0,0.000000,0,2,2,2,3,0,1,2,4
7,J. Hart,8,12,0.666667,3,5,0.600000,1,3,0.333333,0,4,4,1,5,2,2,5,20
8,J. McGee,5,6,0.833333,0,0,0.000000,3,4,0.750000,3,5,8,1,0,3,0,3,13
9,L. Ball,2,7,0.285714,1,4,0.250000,2,2,1.000000,2,2,4,1,4,0,1,3,7
